In [2]:
%%sh
wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


--2021-04-17 12:02:58--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

     0K .......... .......... .......... .......... ..........  1%  637K 7s
    50K .......... .......... .......... .......... ..........  2% 1.14M 6s
   100K .......... .......... .......... .......... ..........  3% 72.6M 4s
   150K .......... .......... .......... .......... ..........  4%  271M 3s
   200K .......... .......... .......... .......... ..........  5% 1.15M 3s
   250K .......... .......... .......... .......... ..........  6% 66.4M 2s
   300K .......... .......... .......... .......... ..........  7%  175M 2s
   350K .......... .......... .......... .......... ..........  8% 1.16M 2s
   400K .......... .......... ......